[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/research-assistant.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239974-lesson-4-research-assistant)

# Research Assistant

## Review

We've covered a few major LangGraph themes:

* Memory
* Human-in-the-loop
* Controllability

Now, we'll bring these ideas together to tackle one of AI's most popular applications: research automation. 

Research is often laborious work offloaded to analysts. AI has considerable potential to assist with this.

However, research demands customization: raw LLM outputs are often poorly suited for real-world decision-making workflows. 

Customized, AI-based [research and report generation](https://jxnl.co/writing/2024/06/05/predictions-for-the-future-of-rag/#reports-over-rag) workflows are a promising way to address this.

## Goal

Our goal is to build a lightweight, multi-agent system around chat models that customizes the research process.

`Source Selection` 
* Users can choose any set of input sources for their research.
  
`Planning` 
* Users provide a topic, and the system generates a team of AI analysts, each focusing on one sub-topic.
* `Human-in-the-loop` will be used to refine these sub-topics before research begins.
  
`LLM Utilization`
* Each analyst will conduct in-depth interviews with an expert AI using the selected sources.
* The interview will be a multi-turn conversation to extract detailed insights as shown in the [STORM](https://arxiv.org/abs/2402.14207) paper.
* These interviews will be captured in a using `sub-graphs` with their internal state. 
   
`Research Process`
* Experts will gather information to answer analyst questions in `parallel`.
* And all interviews will be conducted simultaneously through `map-reduce`.

`Output Format` 
* The gathered insights from each interview will be synthesized into a final report.
* We'll use customizable prompts for the report, allowing for a flexible output format. 

![Screenshot 2024-08-26 at 7.26.33 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbb164d61c93d48e604091_research-assistant1.png)

## Setup

We'll use [LangSmith](https://docs.langchain.com/langsmith/home) for [tracing](https://docs.langchain.com/langsmith/observability-concepts).

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Generate Analysts: Human-In-The-Loop

Create analysts and review them using human-in-the-loop.

In [3]:
import json
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from IPython.display import Image, display

# LangChain & LangGraph
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from json_repair import repair_json

# --- 1. Models ---
class Analyst(BaseModel):
    affiliation: str = Field(description="Primary affiliation of the analyst.")
    name: str = Field(description="Name of the analyst.")
    role: str = Field(description="Role of the analyst in the context of the topic.")
    description: str = Field(description="Description of the analyst focus, concerns, and motives.")

class Perspectives(BaseModel):
    analysts: List[Analyst] = Field(description="List of analysts.")

class GenerateAnalystsState(TypedDict):
    topic: str
    max_analysts: int
    human_analyst_feedback: str
    analysts: List[Analyst]

# --- 2. Ollama Initialization ---
# Qwen handles 'json' mode very well.
llm = ChatOllama(
    model="qwen2.5:8b", 
    temperature=0,
    format="json"  # Constrains the model to output valid JSON
)

# --- 3. Node Logic ---
analyst_instructions = """You are tasked with creating a set of AI analyst personas.
Topic: {topic}
Editorial Feedback: {human_analyst_feedback}
Number of Analysts: {max_analysts}

Pick the top themes and assign one analyst to each. 
Respond ONLY with a JSON object containing an 'analysts' key."""

def create_analysts(state: GenerateAnalystsState):
    topic = state['topic']
    max_analysts = state['max_analysts']
    human_analyst_feedback = state.get('human_analyst_feedback', '') or "No feedback provided."
    
    # Bind structured output
    structured_llm = llm.with_structured_output(Perspectives)

    system_content = analyst_instructions.format(
        topic=topic,
        human_analyst_feedback=human_analyst_feedback, 
        max_analysts=max_analysts
    )

    # The "Nudge": We start the AI's response for it to ensure it stays in JSON.
    messages = [
        SystemMessage(content=system_content),
        HumanMessage(content="Generate the analysts now."),
        AIMessage(content='{"analysts": [') 
    ]

    try:
        # Standard attempt
        response = structured_llm.invoke(messages)
        analyst_list = response.analysts
    except Exception as e:
        print(f"--- Parsing Error: {e}. Attempting Repair... ---")
        # Fallback: Get raw text and use json_repair
        raw_response = llm.invoke(messages)
        repaired = repair_json(raw_response.content)
        data = json.loads(repaired)
        analyst_list = [Analyst(**a) for a in data.get("analysts", [])]
    
    # We clear feedback to prevent an infinite loop later
    return {"analysts": analyst_list, "human_analyst_feedback": None}

def human_feedback(state: GenerateAnalystsState):
    pass

def should_continue(state: GenerateAnalystsState):
    # If the user provides feedback string, go back. 
    # If they hit "OK" or leave it empty, we end.
    feedback = state.get('human_analyst_feedback', None)
    if not feedback or feedback.strip().lower() in ["ok", "approved", "none", ""]:
        return END
    return "create_analysts"

# --- 4. Graph Construction ---
builder = StateGraph(GenerateAnalystsState)
builder.add_node("create_analysts", create_analysts)
builder.add_node("human_feedback", human_feedback)

builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges(
    "human_feedback", 
    should_continue, 
    {"create_analysts": "create_analysts", "__end__": END}
)

memory = MemorySaver()
graph = builder.compile(interrupt_before=['human_feedback'], checkpointer=memory)

# --- 5. Execution ---
thread = {"configurable": {"thread_id": "Harris_1"}}
inputs = {"topic": "The benefits of adopting LangGraph as an agent framework", "max_analysts": 3}

print("--- Running Generation ---")
for event in graph.stream(inputs, thread, stream_mode="values"):
    analysts = event.get('analysts', [])
    if analysts:
        for a in analysts:
            print(f"Name: {a.name}\nRole: {a.role}\n{'-'*20}")

In [ ]:
# Configuration
max_analysts = 3 
topic = "The benefits of adopting LangGraph as an agent framework"
thread = {"configurable": {"thread_id": "Harris_Research_v1"}}

print(f"--- Generating Analysts for: {topic} ---\n")

# Run the graph until the first interruption
for event in graph.stream({"topic": topic, "max_analysts": max_analysts}, thread, stream_mode="values"):
    analysts = event.get('analysts', [])
    if analysts:
        # Clear the previous output if needed, then print the current state
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 50)

# Check if the graph is currently paused
state = graph.get_state(thread)
if state.next:
    print(f"\n[PAUSED] Graph is waiting at node: {state.next[0]}")

In [ ]:
# SET YOUR FEEDBACK HERE
# Use "OK" to finish, or write instructions to change the team.
user_input = "Add a cybersecurity expert who is skeptical of AI agents." 

# 1. Update the state with your feedback
graph.update_state(
    thread, 
    {"human_analyst_feedback": user_input}, 
    as_node="human_feedback"
)

print(f"--- Feedback sent: '{user_input}' ---")

# 2. Resume the stream
for event in graph.stream(None, thread, stream_mode="values"):
    if "analysts" in event:
        print("\nUpdated Analyst Team:")
        for a in event['analysts']:
            print(f" - {a.name} ({a.role})")

# Check if we are done
final_state = graph.get_state(thread)
if not final_state.next:
    print("\n--- Process Complete: Analysts Approved! ---")
else:
    print("\n--- Graph Paused again for further feedback ---")

In [ ]:
# Input
max_analysts = 3 
topic = "The benefits of adopting LangGraph as an agent framework"
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream({"topic":topic,"max_analysts":max_analysts,}, thread, stream_mode="values"):
    # Review
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 50)  

In [ ]:
# Get state and look at next node
state = graph.get_state(thread)
state.next

In [ ]:
# We now update the state as if we are the human_feedback node
graph.update_state(thread, {"human_analyst_feedback": 
                            "Add in someone from a startup to add an entrepreneur perspective"}, as_node="human_feedback")

In [ ]:
# Continue the graph execution
for event in graph.stream(None, thread, stream_mode="values"):
    # Review
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 50) 

In [ ]:
# If we are satisfied, then we simply supply no feedback
further_feedack = None
graph.update_state(thread, {"human_analyst_feedback": 
                            further_feedack}, as_node="human_feedback")

In [ ]:
# Continue the graph execution to end
for event in graph.stream(None, thread, stream_mode="updates"):
    print("--Node--")
    node_name = next(iter(event.keys()))
    print(node_name)

In [ ]:
final_state = graph.get_state(thread)
analysts = final_state.values.get('analysts')

In [ ]:
final_state.next

In [ ]:
for analyst in analysts:
    print(f"Name: {analyst.name}")
    print(f"Affiliation: {analyst.affiliation}")
    print(f"Role: {analyst.role}")
    print(f"Description: {analyst.description}")
    print("-" * 50) 

## Conduct Interview

### Generate Question

The analyst will ask questions to the expert.

In [ ]:
import operator
from typing import  Annotated
from langgraph.graph import MessagesState

class InterviewState(MessagesState):
    max_num_turns: int # Number turns of conversation
    context: Annotated[list, operator.add] # Source docs
    analyst: Analyst # Analyst asking questions
    interview: str # Interview transcript
    sections: list # Final key we duplicate in outer state for Send() API

class SearchQuery(BaseModel):
    # '...' means this field is REQUIRED. 
    # The model is now legally obligated to provide a string.
    search_query: str = Field(..., description="Search query for retrieval.")

In [ ]:
question_instructions = """You are an analyst tasked with interviewing an expert to learn about a specific topic. 

Your goal is boil down to interesting and specific insights related to your topic.

1. Interesting: Insights that people will find surprising or non-obvious.
        
2. Specific: Insights that avoid generalities and include specific examples from the expert.

Here is your topic of focus and set of goals: {goals}
        
Begin by introducing yourself using a name that fits your persona, and then ask your question.

Continue to ask questions to drill down and refine your understanding of the topic.
        
When you are satisfied with your understanding, complete the interview with: "Thank you so much for your help!"

Remember to stay in character throughout your response, reflecting the persona and goals provided to you."""


def generate_question(state: InterviewState):
    """ 
    Node to generate the next interview question.
    Optimized for local LLM persona consistency.
    """

    analyst = state["analyst"]
    messages = state["messages"]

    # 1. System Prompt with clear role boundaries
    # We append specific 'Formatting' instructions to ensure it stays a question.
    system_message_content = (
        f"{question_instructions.format(goals=analyst.persona)}\n\n"
        "IMPORTANT: You are the interviewer. Ask exactly ONE follow-up question. "
        "Do not answer for the expert. Do not summarize."
    )
    
    # 2. Context Window Management
    # Local models on i5s get 'confused' by long histories. 
    # Usually, only the last 3-5 messages are needed to form a great next question.
    recent_messages = messages[-5:] if len(messages) > 5 else messages

    # 3. Invoke the model
    # We wrap it to ensure the model knows it is 'continuing' the interview
    response = llm.invoke([SystemMessage(content=system_message_content)] + recent_messages)
    
    # 4. Consistency Fix: Ensure the message is attributed to the analyst
    # This prevents the 'route_messages' function from miscounting turns.
    response.name = "analyst"
    
    return {"messages": [response]}

### Generate Answer: Parallelization

The expert will gather information from multiple sources in parallel to answer questions.

For example, we can use:

* Specific web sites e.g., via [`WebBaseLoader`](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base)
* Indexed documents e.g., via [RAG](https://docs.langchain.com/oss/python/langchain/retrieval)
* Web search
* Wikipedia search

You can try different web search tools, like [Tavily](https://tavily.com/).

In [ ]:
# Web search tool
from langchain_tavily import TavilySearch  # updated 1.

tavily_search = TavilySearch(max_results=3)

In [ ]:
# Wikipedia search tool
from langchain_community.document_loaders import WikipediaLoader

Now, we create nodes to search the web and wikipedia.

We'll also create a node to answer analyst questions.

Finally, we'll create nodes to save the full interview and to write a summary ("section") of the interview.

In [ ]:
import json
from json_repair import repair_json
from langchain_core.messages import get_buffer_string

# Search query writing
search_instructions = SystemMessage(content=f"""You will be given a conversation between an analyst and an expert. 

Your goal is to generate a well-structured query for use in retrieval and / or web-search related to the conversation.
        
First, analyze the full conversation.

Pay particular attention to the final question posed by the analyst.

Convert this final question into a well-structured web search query""")


def search_web(state: InterviewState):
    """ 
    Final Titanium Version: Specifically designed to strip <think> tags 
    and Alex Carter dialogue before parsing.
    """
    
    # 1. Force the model to be a 'JSON robot'
    search_instructions = SystemMessage(content="Return ONLY JSON. No thinking. No talking.")
    
    try:
        # Optimization: Only give the model the very last instruction
        # to stop it from trying to 'continue' the Alex Carter interview.
        last_user_msg = state['messages'][-1].content
        model_input = [search_instructions, HumanMessage(content=f"Create a search query for: {last_user_msg}")]
        
        # IMPORTANT: Use .invoke() on the raw LLM, NOT the structured_llm.
        # This allows us to manually repair the 'stuttering' output.
        raw_response = llm.invoke(model_input)
        content = raw_response.content
        
        # 2. THE CLEANUP: Remove <think> blocks and extra text
        # This fixes the specific log error you just shared
        cleaned_content = repair_json(content) 
        data = json.loads(cleaned_content)
        query_text = data.get("search_query", last_user_msg)
        
    except Exception as e:
        # FALLBACK: If the model is completely broken, just use the last message content
        print(f"--- Repair Failed: {e}. Using fallback query. ---")
        query_text = state['messages'][-1].content[:100]

    # 3. Sanitize and Search
    clean_query = re.sub(r'[{}":]', '', query_text).strip()
    
    try:
        data = tavily_search.invoke({"query": clean_query})
        search_docs = data.get("results", data) if isinstance(data, dict) else data
    except Exception:
        search_docs = []

    # 4. Format
    formatted_docs = "\n\n---\n\n".join(
        [f'<Document href="{d.get("url", "N/A")}"/>\n{d.get("content", "")}\n</Document>' 
         for d in search_docs if isinstance(d, dict)]
    )

    return {"context": [formatted_search_docs]}


def search_wikipedia(state: InterviewState):
    """ 
    Latest Solution: Advanced recovery using json-repair to handle
    thinking leaks, missing brackets, and local LLM chatter.
    """

    search_instructions = SystemMessage(content=(
        "Generate a concise Wikipedia search query based on the conversation. "
        "Return ONLY a JSON object with a 'search_query' key. "
    ))

    structured_llm = llm.with_structured_output(SearchQuery)
    
    try:
        # Optimization: Last message only + pre-fill opening brace
        model_input = [search_instructions, state['messages'][-1]]
        model_input.append(AIMessage(content='{"search_query": "'))
        
        # We catch the raw response to allow for repair if parsing fails
        response = structured_llm.invoke(model_input)
        query_text = response.search_query
        
    except Exception as e:
        # THE FIX: If Pydantic/Structured_output fails, we repair the string manually
        print(f"--- Parsing Error: {e}. Attempting JSON Repair... ---")
        
        # Get the 'messy' content from the error or the last model attempt
        raw_content = str(e)
        
        # Use json_repair to find and fix the JSON buried in the chatter
        repaired_json = repair_json(raw_content)
        try:
            data = json.loads(repaired_json)
            query_text = data.get("search_query", state['messages'][-1].content[:60])
        except:
            # Absolute fallback to message text
            query_text = state['messages'][-1].content[:60].strip()

    # 2. Query Sanitization
    clean_query = re.sub(r'[??""“”]', '', query_text).strip()

    # 3. Search Wikipedia
    try:
        search_docs = WikipediaLoader(query=clean_query, load_max_docs=2).load()
    except Exception:
        search_docs = []

    # 4. Enhanced Formatting
    formatted_docs_list = [
        f'<Document source="{d.metadata.get("source", "N/A")}"/>\n{d.page_content.replace("\n", " ")}\n</Document>'
        for d in search_docs
    ]

    return {"context": ["\n\n---\n\n".join(formatted_docs_list)]}
    

answer_instructions = """You are an expert being interviewed by an analyst.

Here is analyst area of focus: {goals}. 
        
You goal is to answer a question posed by the interviewer.

To answer question, use this context:
        
{context}

When answering questions, follow these guidelines:
        
1. Use only the information provided in the context. 
        
2. Do not introduce external information or make assumptions beyond what is explicitly stated in the context.

3. The context contain sources at the topic of each individual document.

4. Include these sources your answer next to any relevant statements. For example, for source # 1 use [1]. 

5. List your sources in order at the bottom of your answer. [1] Source 1, [2] Source 2, etc
        
6. If the source is: <Document source="assistant/docs/llama3_1.pdf" page="7"/>' then just list: 
        
[1] assistant/docs/llama3_1.pdf, page 7 
        
And skip the addition of the brackets as well as the Document source preamble in your citation."""


def generate_answer(state: InterviewState):
    """ Node to answer the analyst's question using retrieved context """
    
    analyst = state["analyst"]
    messages = state["messages"]
    context = state["context"]

    # 1. Performance: Limit context window for your i5
    # Joining the last 3 search results into a single string for the prompt
    recent_context_str = "\n\n".join(context[-3:]) if context else "No context available."
    
    # 2. System Instructions
    # We explicitly tell the model it IS the expert to prevent persona drift
    system_content = answer_instructions.format(
        goals=analyst.persona, 
        context=recent_context_str
    )
    
    # 3. Invoke with a sliding window
    # Taking the last 5 messages prevents 'Context Bloat' and keep generation fast
    response = llm.invoke([SystemMessage(content=system_content)] + messages[-5:])
            
    # 4. SPEAKER TRACKING: 
    # We wrap the response in a fresh AIMessage with a name.
    # This ensures your route_messages count (len([m for m in messages if m.name == 'expert']))
    # works reliably across every turn.
    expert_message = AIMessage(content=response.content, name="expert")
    
    return {"messages": [expert_message]}
    

section_writer_instructions = """You are an expert technical writer. 
            
Your task is to create a short, easily digestible section of a report based on a set of source documents.

1. Analyze the content of the source documents: 
- The name of each source document is at the start of the document, with the <Document tag.
        
2. Create a report structure using markdown formatting:
- Use ## for the section title
- Use ### for sub-section headers
        
3. Write the report following this structure:
a. Title (## header)
b. Summary (### header)
c. Sources (### header)

4. Make your title engaging based upon the focus area of the analyst: 
{focus}

5. For the summary section:
- Set up summary with general background / context related to the focus area of the analyst
- Emphasize what is novel, interesting, or surprising about insights gathered from the interview
- Create a numbered list of source documents, as you use them
- Do not mention the names of interviewers or experts
- Aim for approximately 400 words maximum
- Use numbered sources in your report (e.g., [1], [2]) based on information from source documents
        
6. In the Sources section:
- Include all sources used in your report
- Provide full links to relevant websites or specific document paths
- Separate each source by a newline. Use two spaces at the end of each line to create a newline in Markdown.
- It will look like:

### Sources
[1] Link or Document name
[2] Link or Document name

7. Be sure to combine sources. For example this is not correct:

[3] https://ai.meta.com/blog/meta-llama-3-1/
[4] https://ai.meta.com/blog/meta-llama-3-1/

There should be no redundant sources. It should simply be:

[3] https://ai.meta.com/blog/meta-llama-3-1/
        
8. Final review:
- Ensure the report follows the required structure
- Include no preamble before the title of the report
- Check that all guidelines have been followed"""


ef write_section(state: InterviewState):
    """ Node to transform interview and context into a final report section """

    # 1. Extract state
    # We include 'interview' here because the conversation often has better 
    # 'human' insights than the raw search results.
    interview = state["interview"]
    context = state["context"]
    analyst = state["analyst"]
    
    # 2. Clean the Context
    # If context is a list of strings/docs, we join them into one readable block.
    # We take the most relevant snippets to keep the i5 snappy.
    processed_context = "\n\n".join(context) if isinstance(context, list) else context

    # 3. System Instructions
    # We use the analyst's specific description to give the section a unique 'voice'.
    system_message = section_writer_instructions.format(focus=analyst.description)
    
    # 4. The Prompt: Combining raw data + the interview transcript
    # This gives the model the 'facts' (context) and the 'narrative' (interview).
    human_content = (
        f"SOURCE DATA:\n{processed_context}\n\n"
        f"INTERVIEW TRANSCRIPT:\n{interview}\n\n"
        f"TASK: Write a cohesive report section based on the above sources. "
        f"Focus specifically on: {analyst.role}"
    )

    # 5. Invoke (no sliding window here, we need the full picture to write)
    section = llm.invoke([
        SystemMessage(content=system_message),
        HumanMessage(content=human_content)
    ])
                
    # Return as a list so it can be 'added' to the final report
    return {"sections": [section.content]}


# Add nodes and edges 
interview_builder = StateGraph(InterviewState)
interview_builder.add_node("ask_question", generate_question)
interview_builder.add_node("search_web", search_web)
interview_builder.add_node("search_wikipedia", search_wikipedia)
interview_builder.add_node("answer_question", generate_answer)
interview_builder.add_node("save_interview", save_interview)
interview_builder.add_node("write_section", write_section)


# Define the graph flow
interview_builder.add_edge(START, "ask_question")

# Parallel Fan-out: One question triggers two separate searches
interview_builder.add_edge("ask_question", "search_web")
interview_builder.add_edge("ask_question", "search_wikipedia")

# Fan-in: Both searches must complete before the expert answers
interview_builder.add_edge("search_web", "answer_question")
interview_builder.add_edge("search_wikipedia", "answer_question")

# The Conditional Loop: Decide whether to keep interviewing or save
interview_builder.add_conditional_edges(
    "answer_question", 
    route_messages,
    {
        "ask_question": "ask_question", 
        "save_interview": "save_interview"
    }
)

# The Finale: Save the transcript and write the report section
interview_builder.add_edge("save_interview", "write_section")
interview_builder.add_edge("write_section", END)


# Interview 
memory = MemorySaver()
interview_graph = interview_builder.compile(checkpointer=memory).with_config(run_name="Conduct Interviews")

# View
display(Image(interview_graph.get_graph().draw_mermaid_png()))

In [ ]:
# Pick one analyst
analysts[0]

Here, we run the interview passing an index of the llama3.1 paper, which is related to our topic.

In [ ]:
from IPython.display import Markdown
messages = [HumanMessage(f"So you said you were writing an article on {topic}?")]
thread = {"configurable": {"thread_id": "1"}}
interview = interview_graph.invoke({"analyst": analysts[0], "messages": messages, "max_num_turns": 2}, thread)
Markdown(interview['sections'][0])

### Parallelze interviews: Map-Reduce

We parallelize the interviews via the `Send()` API, a map step.

We combine them into the report body in a reduce step.

### Finalize

We add a final step to write an intro and conclusion to the final report.

In [ ]:
import operator
from typing import List, Annotated
from typing_extensions import TypedDict

class ResearchGraphState(TypedDict):
    topic: str # Research topic
    max_analysts: int # Number of analysts
    human_analyst_feedback: str # Human feedback
    analysts: List[Analyst] # Analyst asking questions
    sections: Annotated[list, operator.add] # Send() API key
    introduction: str # Introduction for the final report
    content: str # Content for the final report
    conclusion: str # Conclusion for the final report
    final_report: str # Final report

In [ ]:
from langgraph.types import Send # updated in 1.0
def initiate_all_interviews(state: ResearchGraphState):
    """ This is the "map" step where we run each interview sub-graph using Send API """    

    # Check if human feedback
    human_analyst_feedback=state.get('human_analyst_feedback')
    if human_analyst_feedback:
        # Return to create_analysts
        return "create_analysts"

    # Otherwise kick off interviews in parallel via Send() API
    else:
        topic = state["topic"]
        return [Send("conduct_interview", {"analyst": analyst,
                                           "messages": [HumanMessage(
                                               content=f"So you said you were writing an article on {topic}?"
                                           )
                                                       ]}) for analyst in state["analysts"]]

report_writer_instructions = """You are a technical writer creating a report on this overall topic: 

{topic}
    
You have a team of analysts. Each analyst has done two things: 

1. They conducted an interview with an expert on a specific sub-topic.
2. They write up their finding into a memo.

Your task: 

1. You will be given a collection of memos from your analysts.
2. Think carefully about the insights from each memo.
3. Consolidate these into a crisp overall summary that ties together the central ideas from all of the memos. 
4. Summarize the central points in each memo into a cohesive single narrative.

To format your report:
 
1. Use markdown formatting. 
2. Include no pre-amble for the report.
3. Use no sub-heading. 
4. Start your report with a single title header: ## Insights
5. Do not mention any analyst names in your report.
6. Preserve any citations in the memos, which will be annotated in brackets, for example [1] or [2].
7. Create a final, consolidated list of sources and add to a Sources section with the `## Sources` header.
8. List your sources in order and do not repeat.

[1] Source 1
[2] Source 2

Here are the memos from your analysts to build your report from: 

{context}"""

def write_report(state: ResearchGraphState):
    # Full set of sections
    sections = state["sections"]
    topic = state["topic"]

    # Concat all sections together
    formatted_str_sections = "\n\n".join([f"{section}" for section in sections])
    
    # Summarize the sections into a final report
    system_message = report_writer_instructions.format(topic=topic, context=formatted_str_sections)    
    report = llm.invoke([SystemMessage(content=system_message)]+[HumanMessage(content=f"Write a report based upon these memos.")]) 
    return {"content": report.content}

intro_conclusion_instructions = """You are a technical writer finishing a report on {topic}

You will be given all of the sections of the report.

You job is to write a crisp and compelling introduction or conclusion section.

The user will instruct you whether to write the introduction or conclusion.

Include no pre-amble for either section.

Target around 100 words, crisply previewing (for introduction) or recapping (for conclusion) all of the sections of the report.

Use markdown formatting. 

For your introduction, create a compelling title and use the # header for the title.

For your introduction, use ## Introduction as the section header. 

For your conclusion, use ## Conclusion as the section header.

Here are the sections to reflect on for writing: {formatted_str_sections}"""

def write_introduction(state: ResearchGraphState):
    # Full set of sections
    sections = state["sections"]
    topic = state["topic"]

    # Concat all sections together
    formatted_str_sections = "\n\n".join([f"{section}" for section in sections])
    
    # Summarize the sections into a final report
    
    instructions = intro_conclusion_instructions.format(topic=topic, formatted_str_sections=formatted_str_sections)    
    intro = llm.invoke([instructions]+[HumanMessage(content=f"Write the report introduction")]) 
    return {"introduction": intro.content}

def write_conclusion(state: ResearchGraphState):
    # Full set of sections
    sections = state["sections"]
    topic = state["topic"]

    # Concat all sections together
    formatted_str_sections = "\n\n".join([f"{section}" for section in sections])
    
    # Summarize the sections into a final report
    
    instructions = intro_conclusion_instructions.format(topic=topic, formatted_str_sections=formatted_str_sections)    
    conclusion = llm.invoke([instructions]+[HumanMessage(content=f"Write the report conclusion")]) 
    return {"conclusion": conclusion.content}

def finalize_report(state: ResearchGraphState):
    """ The is the "reduce" step where we gather all the sections, combine them, and reflect on them to write the intro/conclusion """
    # Save full final report
    content = state["content"]
    if content.startswith("## Insights"):
        content = content.strip("## Insights")
    if "## Sources" in content:
        try:
            content, sources = content.split("\n## Sources\n")
        except:
            sources = None
    else:
        sources = None

    final_report = state["introduction"] + "\n\n---\n\n" + content + "\n\n---\n\n" + state["conclusion"]
    if sources is not None:
        final_report += "\n\n## Sources\n" + sources
    return {"final_report": final_report}

# Add nodes and edges 
builder = StateGraph(ResearchGraphState)
builder.add_node("create_analysts", create_analysts)
builder.add_node("human_feedback", human_feedback)
builder.add_node("conduct_interview", interview_builder.compile())
builder.add_node("write_report",write_report)
builder.add_node("write_introduction",write_introduction)
builder.add_node("write_conclusion",write_conclusion)
builder.add_node("finalize_report",finalize_report)

# Logic
builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges("human_feedback", initiate_all_interviews, ["create_analysts", "conduct_interview"])
builder.add_edge("conduct_interview", "write_report")
builder.add_edge("conduct_interview", "write_introduction")
builder.add_edge("conduct_interview", "write_conclusion")
builder.add_edge(["write_conclusion", "write_report", "write_introduction"], "finalize_report")
builder.add_edge("finalize_report", END)

# Compile
memory = MemorySaver()
graph = builder.compile(interrupt_before=['human_feedback'], checkpointer=memory)
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

Let's ask an open-ended question about LangGraph.

In [ ]:
# Inputs
max_analysts = 3 
topic = "The benefits of adopting LangGraph as an agent framework"
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream({"topic":topic,
                           "max_analysts":max_analysts}, 
                          thread, 
                          stream_mode="values"):
    
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 50)  

In [ ]:
# We now update the state as if we are the human_feedback node
graph.update_state(thread, {"human_analyst_feedback": 
                                "Add in the CEO of gen ai native startup"}, as_node="human_feedback")

In [ ]:
# Check
for event in graph.stream(None, thread, stream_mode="values"):
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 50)  

In [ ]:
# Confirm we are happy
graph.update_state(thread, {"human_analyst_feedback": 
                            None}, as_node="human_feedback")

In [ ]:
# Continue
for event in graph.stream(None, thread, stream_mode="updates"):
    print("--Node--")
    node_name = next(iter(event.keys()))
    print(node_name)

In [ ]:
from IPython.display import Markdown
final_state = graph.get_state(thread)
report = final_state.values.get('final_report')
Markdown(report)

We can look at the trace:

https://smith.langchain.com/public/2933a7bb-bcef-4d2d-9b85-cc735b22ca0c/r